In [36]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id
import pseudopeople as pseudo

print("Loading Spark")
spark = SparkSession.builder.appName("Record Linkage").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()


Loading Spark


In [37]:
print("Generating Pseudopeople Data")

df_1 = spark.read.csv("/home/Ankith/RecordLinkage_Spark/Record_Linkage_Spark/src/pse_sample4.1.1.csv", header=True, sep='\t')
df_2 = spark.read.csv("/home/Ankith/RecordLinkage_Spark/Record_Linkage_Spark/src/pse_sample4.1.2.csv", header=True, sep='\t')

Generating Pseudopeople Data


In [38]:
df_1.sort(col("last_name"),col("first_name"),col("middle_initial"),col("age"),col("street_name")).show(5)
df_1 = df_1.show(5)

+------------+----------+--------------+---------+---+-------------+-------------+-------------+------------+----------+-----+-------+----------------------------+------+--------------+
| simulant_id|first_name|middle_initial|last_name|age|date_of_birth|street_number|  street_name| unit_number|      city|state|zipcode|relation_to_reference_person|   sex|race_ethnicity|
+------------+----------+--------------+---------+---+-------------+-------------+-------------+------------+----------+-----+-------+----------------------------+------+--------------+
|9402_1006023|   Allisom|             M|        A|  0|   05/04/2019|         2415|    dale st s|office # 308|providence|   RI|  02833|            Biological child|Female|         White|
|9402_1006023|   Allison|             M|        A|  0|   05/04/2019|         2415|    dale st s|office # 308|providence|   RI|  02833|            Biological child|Female|         White|
|1299_1016030|    Alyssa|             M|        A| 18|   01/16/2002|  

In [45]:
df_2.show(5)

+-----------+----------+--------------+---------+---+-------------+-------------+------------+-----------+------------+-----+-------+----------------------------+------+--------------+
|simulant_id|first_name|middle_initial|last_name|age|date_of_birth|street_number| street_name|unit_number|        city|state|zipcode|relation_to_reference_person|   sex|race_ethnicity|
+-----------+----------+--------------+---------+---+-------------+-------------+------------+-----------+------------+-----+-------+----------------------------+------+--------------+
|3298_700403|    Joshua|             H|     Nava| 20|   04/21/1999|          nan|    pansy rd|        nan|  portsmouth|   RI|  02852|        Noninstitutionali...|  Male|         White|
|4943_978644|   William|             C|  Acevedo| 50|   03/18/1970|         1710|flying a trl|        nan|         nan|   RI|  02864|        Noninstitutionali...|  Male|        Latino|
|  734_60972|     Kenny|             J|   Gatian| 32|   06/14/1988|        

In [33]:

#dropping duplicates from df_
df_1_nd = df_1.dropDuplicates(['simulant_id'])
df_1_nd.orderBy(["simulant_id"]).show()


+------------+----------+--------------+----------+---+-------------+-------------+-------------------+-----------+---------------+-----+-------+----------------------------+------+--------------+
| simulant_id|first_name|middle_initial| last_name|age|date_of_birth|street_number|        street_name|unit_number|           city|state|zipcode|relation_to_reference_person|   sex|race_ethnicity|
+------------+----------+--------------+----------+---+-------------+-------------+-------------------+-----------+---------------+-----+-------+----------------------------+------+--------------+
|1007_1001056|      Judy|             C| Zimmerman| 31|   10/14/1988|         1957|           ctrl ave|        nan|       scituate|   RI|  02895|              Other relative|Female|         Asian|
|1007_1011208|     Milan|             E|    Potter|  0|   08/28/2019|          177|    st clements ave|        nan|      pawtucket|   RI|  02807|                  Grandchild|   nan|         White|
|1007_1015136| 

In [31]:

#Dropping Duplicates from df2
df_2_nd = df_2.dropDuplicates(['simulant_id'])
df_2_nd.orderBy(col("simulant_id")).show(50)



+------------+-----------+--------------+---------------+---+-------------+-------------+-------------------+-----------+----------------+-----+-------+----------------------------+------+--------------------+
| simulant_id| first_name|middle_initial|      last_name|age|date_of_birth|street_number|        street_name|unit_number|            city|state|zipcode|relation_to_reference_person|   sex|      race_ethnicity|
+------------+-----------+--------------+---------------+---+-------------+-------------+-------------------+-----------+----------------+-----+-------+----------------------------+------+--------------------+
|1007_1001056|       Judy|             C|      Zimmerman| 31|   10/14/1988|         1957|           ctrl ave|        nan|        scituate|   RI|  02895|              Other relative|Female|               Asian|
|1007_1011208|      Milan|             E|         Potter|  0|   08/28/2019|          177|    st clements ave|        nan|       pawtucket|   RI|  02807|        